# Day 6: Visualization & Communication
## Notebook 3: Interactive Visualization with Plotly

**Introduction to Scientific Programming**  
CNC-UC | 2025

This notebook covers:
- Understand when interactivity adds value
- Create interactive plots with Plotly Express
- Add hover information and interactive controls
- Create 3D visualizations for high-dimensional data
- Export interactive plots as HTML for sharing

In [2]:
# Import required libraries
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set random seed
np.random.seed(42)

print("✓ Plotly loaded successfully")
print(f"  Version: {px.__version__ if hasattr(px, '__version__') else 'Unknown'}")

✓ Plotly loaded successfully
  Version: Unknown


## Part 1: When to Use Interactive Plots?

**Use interactive plots for:**
- Data exploration (zoom, pan, hover for details)
- Large datasets with many points
- High-dimensional data (3D, multiple variables)
- Web sharing and dashboards
- Presentations with live data

**Avoid for:**
- Journal publications (use static Matplotlib/Seaborn)
- Print-only documents
- Simple plots that don't benefit from interactivity

## Part 2: Plotly Express Basics

Plotly Express provides a high-level interface for quick interactive plots.

In [3]:
# Generate sample neuroscience data
n_trials = 200

data = pd.DataFrame({
    'trial': range(n_trials),
    'reaction_time': 400 + 80 * np.random.randn(n_trials) + np.arange(n_trials) * 0.2,
    'accuracy': np.clip(0.5 + np.arange(n_trials) * 0.002 + np.random.randn(n_trials) * 0.1, 0, 1),
    'condition': np.random.choice(['Control', 'Treatment'], n_trials),
    'subject': np.repeat([f'S{i+1}' for i in range(10)], n_trials // 10),
    'session': np.tile(range(1, 21), n_trials // 20)
})

# Add some neural activity data
data['neural_activity'] = 10 + 5 * np.sin(2 * np.pi * data['trial'] / 50) + np.random.randn(n_trials) * 2

print("Sample data:")
print(data.head())
print(f"\nShape: {data.shape}")

Sample data:
   trial  reaction_time  accuracy  condition subject  session  neural_activity
0      0     439.737132  0.535779    Control      S1        1        13.289935
1      1     389.138856  0.558078    Control      S1        2        10.128594
2      2     452.215083  0.612305    Control      S1        3        12.396563
3      3     522.442389  0.611380    Control      S1        4        12.463123
4      4     382.067730  0.370233  Treatment      S1        5        18.566530

Shape: (200, 7)


In [7]:
# Interactive scatter plot
fig = px.scatter(data, 
                 x='trial', 
                 y='reaction_time',
                 color='condition',
                 size='accuracy',
                 hover_data=['subject', 'session', 'neural_activity'],
                 title='Reaction Time Over Trials',
                 labels={'reaction_time': 'Reaction Time (ms)', 'trial': 'Trial Number'})

fig.update_layout(height=500)
fig.show()

print("\n💡 Try hovering over points, zooming, and panning!")


💡 Try hovering over points, zooming, and panning!


## Part 3: Common Interactive Plot Types

In [6]:
# Line plot with multiple traces
# Aggregate by subject and session
agg_data = data.groupby(['subject', 'session', 'condition']).agg({
    'reaction_time': 'mean',
    'accuracy': 'mean'
}).reset_index()

fig = px.line(agg_data, 
              x='session', 
              y='accuracy',
              color='subject',
              line_group='subject',
              facet_col='condition',
              title='Learning Curves by Subject and Condition',
              labels={'accuracy': 'Mean Accuracy', 'session': 'Session Number'})

fig.update_yaxes(range=[0, 1])
fig.show()

print("\n💡 Click legend items to show/hide subjects!")


💡 Click legend items to show/hide subjects!


In [7]:
# Box plot with individual points
fig = px.box(data, 
             x='condition', 
             y='reaction_time',
             color='condition',
             points='all',  # Show all points
             hover_data=['subject', 'trial'],
             title='Reaction Time Distribution by Condition')

fig.update_layout(showlegend=False, height=500)
fig.show()

print("\n💡 Hover over individual points to see details!")


💡 Hover over individual points to see details!


In [6]:
# Histogram with overlapping distributions
fig = px.histogram(data, 
                   x='reaction_time',
                   color='condition',
                   marginal='box',  # Add box plot on top
                   nbins=30,
                   opacity=0.6,
                   title='RT Distribution with Marginal Box Plot')

fig.update_layout(barmode='overlay', height=500)
fig.show()

## Part 4: 3D Visualization for High-Dimensional Data

In [9]:
# Generate 3D neural data (e.g., from dimensionality reduction)
n_points = 300
clusters = 3

neural_3d = pd.DataFrame()
for i in range(clusters):
    cluster_data = pd.DataFrame({
        'PC1': np.random.randn(n_points // clusters) + i * 3,
        'PC2': np.random.randn(n_points // clusters) + i * 2,
        'PC3': np.random.randn(n_points // clusters) + i * 1.5,
        'cluster': f'Cluster {i+1}',
        'time_point': range(n_points // clusters)
    })
    neural_3d = pd.concat([neural_3d, cluster_data], ignore_index=True)

# 3D scatter plot
fig = px.scatter_3d(neural_3d,
                    x='PC1', y='PC2', z='PC3',
                    color='cluster',
                    size_max=10,
                    opacity=0.7,
                    title='Neural Population Activity in 3D (PCA)',
                    labels={'PC1': 'Principal Component 1',
                           'PC2': 'Principal Component 2',
                           'PC3': 'Principal Component 3'})

fig.update_layout(height=600)
fig.show()

print("\n💡 Rotate, zoom, and explore the 3D space!")


💡 Rotate, zoom, and explore the 3D space!


In [10]:
# 3D trajectory (e.g., neural state evolution over time)
fig = px.line_3d(neural_3d,
                 x='PC1', y='PC2', z='PC3',
                 color='cluster',
                 line_group='cluster',
                 title='Neural State Trajectories Over Time')

fig.update_traces(line=dict(width=3))
fig.update_layout(height=600)
fig.show()

## Part 5: Heatmaps and Correlation Matrices

In [11]:
# Create correlation matrix
numeric_cols = ['trial', 'reaction_time', 'accuracy', 'neural_activity']
correlation = data[numeric_cols].corr()

# Interactive heatmap
fig = px.imshow(correlation,
                text_auto='.2f',
                color_continuous_scale='RdBu_r',
                zmin=-1, zmax=1,
                title='Correlation Matrix (Interactive)',
                labels=dict(color='Correlation'))

fig.update_layout(height=500)
fig.show()

print("\n💡 Hover to see exact correlation values!")


💡 Hover to see exact correlation values!


In [12]:
# Time-frequency heatmap (simulated spectrogram)
time = np.linspace(0, 10, 100)
frequencies = np.linspace(1, 50, 50)
spectrogram = np.random.randn(50, 100) + np.sin(2 * np.pi * frequencies[:, np.newaxis] * time / 10)

fig = px.imshow(spectrogram,
                x=time,
                y=frequencies,
                color_continuous_scale='Viridis',
                aspect='auto',
                title='Neural Activity Spectrogram',
                labels={'x': 'Time (s)', 'y': 'Frequency (Hz)', 'color': 'Power'})

fig.update_layout(height=500)
fig.show()

## Part 6: Subplots and Multiple Panels

In [ ]:
# Create subplots with plotly.graph_objects
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Reaction Time Over Trials', 'Accuracy Over Trials',
                    'RT Distribution', 'RT vs Accuracy'),
    specs=[[{'type': 'scatter'}, {'type': 'scatter'}],
           [{'type': 'histogram'}, {'type': 'scatter'}]]
)

# Panel 1: RT over trials
for condition in data['condition'].unique():
    subset = data[data['condition'] == condition]
    fig.add_trace(
        go.Scatter(x=subset['trial'], y=subset['reaction_time'],
                   mode='markers', name=condition, legendgroup=condition,
                   marker=dict(size=4, opacity=0.6)),
        row=1, col=1
    )

# Panel 2: Accuracy over trials
for condition in data['condition'].unique():
    subset = data[data['condition'] == condition]
    fig.add_trace(
        go.Scatter(x=subset['trial'], y=subset['accuracy'],
                   mode='markers', name=condition, legendgroup=condition,
                   showlegend=False, marker=dict(size=4, opacity=0.6)),
        row=1, col=2
    )

# Panel 3: RT histogram
for condition in data['condition'].unique():
    subset = data[data['condition'] == condition]
    fig.add_trace(
        go.Histogram(x=subset['reaction_time'], name=condition,
                     legendgroup=condition, showlegend=False, opacity=0.6),
        row=2, col=1
    )

# Panel 4: RT vs Accuracy
for condition in data['condition'].unique():
    subset = data[data['condition'] == condition]
    fig.add_trace(
        go.Scatter(x=subset['accuracy'], y=subset['reaction_time'],
                   mode='markers', name=condition, legendgroup=condition,
                   showlegend=False, marker=dict(size=4, opacity=0.6)),
        row=2, col=2
    )

# Update axes labels
fig.update_xaxes(title_text='Trial', row=1, col=1)
fig.update_xaxes(title_text='Trial', row=1, col=2)
fig.update_xaxes(title_text='Reaction Time (ms)', row=2, col=1)
fig.update_xaxes(title_text='Accuracy', row=2, col=2)

fig.update_yaxes(title_text='RT (ms)', row=1, col=1)
fig.update_yaxes(title_text='Accuracy', row=1, col=2)
fig.update_yaxes(title_text='Count', row=2, col=1)
fig.update_yaxes(title_text='RT (ms)', row=2, col=2)

fig.update_layout(height=700, title_text='Multi-Panel Interactive Dashboard')
fig.show()

## Part 7: Animation for Time Series Data

In [14]:
# Create animated scatter plot showing evolution over sessions
anim_data = data.copy()
anim_data['session'] = (anim_data['trial'] // 10) + 1

fig = px.scatter(anim_data,
                 x='neural_activity',
                 y='reaction_time',
                 color='condition',
                 size='accuracy',
                 animation_frame='session',
                 animation_group='subject',
                 hover_data=['subject'],
                 range_x=[0, 20],
                 range_y=[200, 600],
                 title='Neural Activity vs Reaction Time Over Sessions')

fig.update_layout(height=600)
fig.show()

print("\n💡 Press play to see how the data evolves over sessions!")


💡 Press play to see how the data evolves over sessions!


## Part 8: Exporting Interactive Plots

In [5]:
# Create a sample interactive figure
fig = px.scatter(data,
                 x='trial',
                 y='reaction_time',
                 color='condition',
                 size='accuracy',
                 hover_data=['subject', 'neural_activity'],
                 title='Interactive Reaction Time Plot')

# Export as HTML (fully interactive)
fig.write_html('./interactive_plot.html')
print("✓ Saved as HTML: ./interactive_plot.html")
print("  → Open in browser for full interactivity")
print("  → Share with colleagues via email or web hosting")

# Export as static image (requires kaleido)
try:
    fig.write_image('./static_plot.png', width=1200, height=600)
    print("\n✓ Saved as PNG: ./static_plot.png")
    print("  → Use for presentations or documents")
except:
    print("\n⚠ To export static images, install: pip install kaleido")

fig.show()

✓ Saved as HTML: ./interactive_plot.html
  → Open in browser for full interactivity
  → Share with colleagues via email or web hosting

⚠ To export static images, install: pip install kaleido


## Part 9: Customization and Styling

In [18]:
# Highly customized figure
fig = px.scatter(data,
                 x='trial',
                 y='reaction_time',
                 color='condition',
                 title='Customized Interactive Plot')

# Update layout for publication-like appearance
fig.update_layout(
    template='plotly_white',  # Clean white background
    font=dict(family='Arial', size=12),
    title=dict(font=dict(size=16, color='#333')),
    xaxis=dict(
        title='Trial Number',
        showgrid=True,
        gridcolor='lightgray',
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        title='Reaction Time (ms)',
        showgrid=True,
        gridcolor='lightgray',
        linecolor='black',
        linewidth=2
    ),
    legend=dict(
        title='Condition',
        orientation='v',
        yanchor='top',
        y=1,
        xanchor='left',
        x=1.02
    ),
    height=500
)

# Update traces (markers)
fig.update_traces(
    marker=dict(size=6, opacity=0.7, line=dict(width=0.5, color='white'))
)

fig.show()

print("\n✓ Customized for cleaner, publication-like appearance")


✓ Customized for cleaner, publication-like appearance


## Summary

### Key Takeaways:

1. **Plotly Express**: High-level interface for quick interactive plots
2. **Interactive features**: Hover, zoom, pan, click, export built-in
3. **Common plots**: Scatter, line, box, histogram, heatmap, 3D
4. **3D visualization**: Essential for high-dimensional data exploration
5. **Subplots**: Use `make_subplots()` for multi-panel layouts
6. **Animation**: Show temporal evolution with `animation_frame`
7. **Export**: HTML for interactivity, PNG/PDF for static use

### When to Use Plotly:
- ✓ Data exploration (large datasets)
- ✓ Web sharing and dashboards
- ✓ High-dimensional data (3D, many variables)
- ✓ Presentations with live data
- ✓ Temporal data with animation

### When NOT to Use Plotly:
- ✗ Journal publications (use Matplotlib/Seaborn)
- ✗ Print-only documents
- ✗ Simple plots where interactivity doesn't add value
- ✗ When precise control over every pixel is needed

### Best Practices:
- Start with Plotly Express for simplicity
- Use `go.Figure()` and `make_subplots()` for complex layouts
- Add meaningful hover data
- Customize for cleaner appearance with `update_layout()`
- Export HTML for colleagues to explore interactively